In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Jan 19 09:49:17 2018

@author: vitor
"""

import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import nltk
from nltk.corpus import stopwords
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
#print('x' in np.arange(5))
#leitura
text = pd.read_csv('chennai_reviews.csv', sep=',',skip_blank_lines=True)
train, test = train_test_split(text, test_size=0.2)#Usando 80% do data base
vetor_atributo=[]#Vetor contendo as palavras usadas
matrix=pd.Series.as_matrix(train)
#Utilizo todos os adjetivos presentes no train.Review_Text segundo o nltk

vetor_frases=[]
for i in range(len(matrix)):#Criando o vetor com todas as frases
    auxx=''
    for j in range(len(matrix[i])):
        if(isinstance(matrix[i][j], str)):
            if(len(matrix[i][j])> len(auxx)):
                auxx=matrix[i][j]
    auxx=re.sub(r"[^\w\s]", '', auxx)#retirando os underscors
    auxx=auxx.lower()
    vetor_frases=np.append(vetor_frases,auxx)

#Vetor de palavras
for i in range(len(vetor_frases)):#Criando o vetor com atributos
    frase=vetor_frases[i]
    if isinstance(frase, str):
        frase= re.sub(r"[^\w\s]", '', frase)#retirando os underscors
        frase=frase.lower()
        palavras = set(frase.split())
        palavras=set(nltk.word_tokenize(frase))
        english_stop_words = set(stopwords.words('english'))
        palavras_filtradas = palavras - english_stop_words
        tagged = nltk.pos_tag(palavras_filtradas)
        aux=np.asmatrix(tagged)
        aux1=np.where(aux=='JJ')#Pegando os adjetivos
        if (len(np.where(aux=='JJ')[0])>0):
            if not(tagged[aux1[0][0]][aux1[1][0]-1] in vetor_atributo):
                vetor_atributo=np.append(vetor_atributo,tagged[aux1[0][0]][aux1[1][0]-1])
#        aux1=np.where(aux=='RBS')
#        if (np.sum(aux1)>0):
#            if not(tagged[aux1[0][0]][aux1[1][0]-1] in vetor_atributo):
#                vetor_atributo=np.append(vetor_atributo,tagged[aux1[0][0]][aux1[1][0]-1])
#        aux1=np.where(aux=='JJS')
#        if (np.sum(aux1)>0): 
#            if not(tagged[aux1[0][0]][aux1[1][0]-1] in vetor_atributo):
#                vetor_atributo=np.append(vetor_atributo,tagged[aux1[0][0]][aux1[1][0]-1])
#            

#Valor de cada uma das porcentages dos sentimentos
p1=0#porcentagem de 1
p2=0#porcentagem de 2
p3=0#porcentagem de 3
error=[]#Vetor que irá conter os indices na matrix de alguma frase que gere error
#for i in range(3813):
#   if(matrix[i][3]=='1' or matrix[i][4]=='1'):
#       p1=p1+1
#   elif(matrix[i][3]=='2' or matrix[i][4]=='2'):
#       p2=p2+1
#   elif(matrix[i][3]=='3'or matrix[i][4]=='3'):
#       p3=p3+1
#   else:
#       error=np.append(error,i)
##for i in range(len(error)-1):
##   if(len(np.where(matrix[int(error[0])]==1)[0])>0):
##       p1=p1+1
##   elif(len(np.where(matrix[int(error[0])]==2)[0])>0):
##       p2=p2+1
##   elif(len(np.where(matrix[int(error[0])]==3)[0])>0):
##       p3=p3+1
v=[]#Vetor com os setimentos
for i in range(len(matrix)):#Criando o vetor com os sentimentos de cada frase
   if(len(np.where(matrix[i]=='1')[0])>0 or len(np.where(matrix[i]==1)[0])>0):
       v=np.append(v,int(1))
   elif(len(np.where(matrix[i]=='2')[0])>0 or len(np.where(matrix[i]==2)[0])>0):
       v=np.append(v,int(2))
   elif(len(np.where(matrix[i]=='3')[0])>0 or len(np.where(matrix[i]==3)[0])>0):
       v=np.append(v,int(3))
   else:
       error=np.append(error,i)
       
p1=len(np.where(v==1)[0])
p2=len(np.where(v==2)[0])
p3=len(np.where(v==3)[0])
aux=[p1,p2,p3]
p1=p1/np.sum(aux)
p2=p2/np.sum(aux)
p3=p3/np.sum(aux)
aux=[p1,p2,p3]#Vetor probabilidade do sentimentos

#matriz binária
#vetor_frases=np.asarray(train.Review_Text)
vetor_frases=[]
for i in range(len(matrix)):#Criando o vetor com todas as frases
    auxx=''
    for j in range(len(matrix[i])):
        if(isinstance(matrix[i][j], str)):
            if(len(matrix[i][j])> len(auxx)):
                auxx=matrix[i][j]
    auxx=re.sub(r"[^\w\s]", '', auxx)
    auxx=auxx.lower()
    vetor_frases=np.append(vetor_frases,auxx)
            
            
            

matriz_binaria=[]

for i in range(len(vetor_frases)):#Criando uma matriz com vetores binarios em relação as
    vetor_aux=[]
    if isinstance(vetor_frases[i], str) and not(len(np.where(error==i)[0])) :    
        for j in range(len(vetor_atributo)):
            if(vetor_atributo[j] in vetor_frases[i].lower()):
                vetor_aux=np.append(vetor_aux,int(1))
            else:
                vetor_aux=np.append(vetor_aux,int(0))
        if(i!=0):
            matriz_binaria=np.vstack([matriz_binaria,vetor_aux])
        else:
            matriz_binaria=vetor_aux
    else:
        error=np.append(error,i)
    
#Learning
vetor_resultado=np.zeros([3,len(vetor_atributo)])
maux=matriz_binaria.transpose()

for i in range(len(maux)):
    aux31=1#Quandtidade de vezes que o atributo[i]=1 aparece quando o valor do sentimento=3
    aux11=1#Quandtidade de vezes que o atributo[i]=1 aparece quando o valor do sentimento=1
    aux21=1#Quandtidade de vezes que o atributo[i]=1 aparece quando o valor do sentimento=2

    perror=0#algumas frases podem não ser analisadas então é necessario ter a quantidade de erros acumulativos em cada frase
#    for j in range(len(maux[0])):
#        if(j in error):
#            perror=perror+1
#        if(v[j+perror]==1 and maux[i][j]==1 and not(j in error)):
#            aux1=aux11+1
#        elif(v[j+perror]==2 and maux[i][j]==1 and not(j in error)):
#            aux2=aux21+1
#        elif(v[j+perror]==3 and maux[i][j]==1 and not(j in error)):
#            aux3=aux31+1  
    
    for j in range(len(maux[0])):
        if(j in error):
            perror=perror+1
        if(v[j+perror]==1 and maux[i][j]==1 and not(j in error)):
            aux11=aux11+1
        if(v[j+perror]==2 and maux[i][j]==1 and not(j in error)):
            aux21=aux21+1
        if(v[j+perror]==3 and maux[i][j]==1 and not(j in error)):
            aux31=aux31+1

#    aux10=aux10/len(np.where(maux[i]==0)[0])
#    aux20=aux20/len(np.where(maux[i]==0)[0])
#    aux30=aux10/len(np.where(maux[i]==0)[0])
#    aux11=aux10/len(np.where(maux[i]==1)[0])
#    aux21=aux20/len(np.where(maux[i]==1)[0])
#    aux31=aux10/len(np.where(maux[i]==1)[0])
    vetor_resultado[0][i]=(aux11/(2+len(np.where(v==1)[0])))
    vetor_resultado[1][i]=(aux21/(2+len(np.where(v==2)[0])))
    vetor_resultado[2][i]=(aux31/(2+len(np.where(v==3)[0])))


 
#test
nmatrix=pd.Series.as_matrix(test)
nvetor_frases=[]
for i in range(len(nmatrix)):
    auxx=''
    for j in range(len(nmatrix[i])):
        if(isinstance(nmatrix[i][j], str)):
            if(len(nmatrix[i][j])> len(auxx)):
                auxx=nmatrix[i][j]
    auxx=re.sub(r"[^\w\s]", '', auxx)
    auxx=auxx.lower()
    nvetor_frases=np.append(nvetor_frases,auxx) 
nv=[]#Vetor de sentimentos do data test
for i in range(len(nmatrix)):
   if(len(np.where(nmatrix[i]=='1')[0])>0 or len(np.where(nmatrix[i]==1)[0])>0):
       nv=np.append(nv,int(1))
   elif(len(np.where(nmatrix[i]=='2')[0])>0 or len(np.where(nmatrix[i]==2)[0])>0):
       nv=np.append(nv,int(2))
   elif(len(np.where(nmatrix[i]=='3')[0])>0 or len(np.where(nmatrix[i]==3)[0])>0):
       nv=np.append(nv,int(3))
final=[]#Vetor binario dizendo se o algoritimo acertou
final_result=[]#Resultado calculado de cada probabilidade
fi=[]
for z in range(len(nvetor_frases)):
    string_test=nvetor_frases[z]#Texto para teste
    vetor_str_test=[]#Vetor auxiliar binario da frase em relação aos atributos
    result=0
    test_result=np.zeros([3,1])
       
    for j in range(len(vetor_atributo)):#Criando o vetor_str_test
        if(vetor_atributo[j] in nvetor_frases[i].lower()):
            vetor_str_test=np.append(vetor_str_test,int(1))
        else:
            vetor_str_test=np.append(vetor_str_test,int(0))
    
    for i in range(len(vetor_resultado)):#Fazendo o calculo da probabilidade de ser cada um dos sentimentos
        result=math.log10(aux[i])
        for j in range(len(vetor_resultado[i])):
            result=result+math.log10(vetor_str_test[j]*(vetor_resultado[i][j])+(1-vetor_str_test[j])*((1-vetor_resultado[i][j])))#]Modelo de Bernoulli
        test_result[i][0]=result
        
    aux4=np.where(np.max(test_result)==test_result)[0]==(nv[z]-1)#Teste para ver se o valor encontrado pelo algoritimo é o mesmo do data test
    final=np.append(final,aux4[0])
    fi=np.append(fi,z)
    final_result=np.append(final_result,test_result)
    
raiting=len(np.where(final==True)[0])/len(final)#Taxa de acerto   
final_result=np.reshape(final_result,[3,len(nvetor_frases)])










LookupError: 
**********************************************************************
  Resource 'tokenizers/punkt/english.pickle' not found.  Please
  use the NLTK Downloader to obtain the resource:  >>>
  nltk.download()
  Searched in:
    - 'C:\\Users\\jeffe/nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - 'C:\\Users\\jeffe\\Anaconda3\\nltk_data'
    - 'C:\\Users\\jeffe\\Anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\jeffe\\AppData\\Roaming\\nltk_data'
    - ''
**********************************************************************